# Query API

Query API for **{target_app}**:

    - linkedService
        - ServiceType
            - redis
            - postgres
    - mounts (directory mappings)
        - targetDir (mapping to) 
        - hostDir (mapping from)
    - environmentVariables
        - name
        - value

In [ ]:
from gql import  gql, Client
from gql.transport.requests import RequestsHTTPTransport


dash_enterprise_host = "dash-playground.plotly.host"
username = "developers"
username_api_key = "faBhA8WwjuLpC8QoEulU"
new_app = ""
target_app = "aa-chris"
# target_app = "aa-tngo-oil-and-gas"
# target_app = "review-app-2"
target_app_config = {}


transport = RequestsHTTPTransport(
    url =  "https://{dash_enterprise_host}/Manager/graphql".format(dash_enterprise_host = dash_enterprise_host), 
    auth = (username, username_api_key), 
    use_json = True
)

client = Client(transport = transport)

# Querying target app...
print(f"Querying {target_app}...", end=" ")

query_string = ("""
{{
    apps(name: "{target_app}", allApps:true) {{
        apps {{
            linkedServices1 {{
                serviceType
            }}
            mounts {{
                targetDir
                hostDir
            }}
            environmentVariables {{
                name
                value
            }}
        }}
    }}
}}""".format(target_app = target_app))

try:
   api_call = client.execute(gql(query_string)) 
except Exception as e: # work on python 3.x
    api_call.error('Failed to upload to ftp: '+ str(e))
# api_call = client.execute(gql(query_string))
api_call_results = []
api_call_results = api_call["apps"]["apps"]
####################################################

## Exception Handling
# if 'addApp' in api_call and 'error' in api_call['addApp']:
#     if api_call['addApp']['error'] != 'error': 'An app with this name already exists in this Dash Server. Please choose a different name.':
#         raise Exception(api_call)
try:
   api_call = client.execute(gql(query_string)) 
except BaseException as e: # work on python 3.x
    logger.error(': '+ str(e))

####################################################
if len(api_call_results) != 0:
    print("OK")
    app_linkedServices = api_call_results[0]["linkedServices"]
    app_mounts = api_call_results[0]["mounts"]
    app_environmentVariables = api_call_results[0]["environmentVariables"]

    app_linkedServices_serviceType = []
    print("Parsing LinkedServices...", end=" ")
    if len(app_linkedServices) != 0:
        print("OK")
        for i in range(len(app_linkedServices)):
            app_linkedServices_serviceType.append(app_linkedServices[i]["serviceType"])
            print("> ", app_linkedServices[i]["serviceType"])
    else:
        print("NULL")

    app_mounts_dict = dict()
    app_mounts_hostDir = []
    app_mounts_targetDir = []
    print("Parsing Mounts...", end=" ")
    if len(app_mounts) != 0:
        print("OK")
        for i in range(len(app_mounts)):
            app_mounts_hostDir.append(app_mounts[i]["hostDir"])
            app_mounts_targetDir.append(app_mounts[i]["targetDir"])
        app_mounts_dict = dict(zip(app_mounts_hostDir, app_mounts_targetDir))   
        for k,v in app_mounts_dict.items():
            print("> ", k, " : ", v)
    else:
        print("NULL")

    app_environmentVariables_filter = [
        'DASH_APP_NAME', 
        'DASH_DOMAIN_BASE', 
        'DASH_LOGOUT_URL', 
        'DASH_PATH_ROUTING', 
        'DASH_SECRET_KEY', 
        'DASH_STREAMBED_DIRECT_IP', 
        'DATABASE_URL', 
        'DOKKU_APP_RESTORE', 
        'DOKKU_APP_TYPE', 
        'DOKKU_PROXY_PORT', 
        'DOKKU_PROXY_PORT_MAP', 
        'GIT_REV', 
        'REDIS_URL', 
        'SCRIPT_NAME', 
        'NO_VHOST'
    ]

    app_environmentVariables_dict = dict()
    app_environmentVariables_name = []
    app_environmentVariables_value = []
    print("Parsing EnvironmentVariables...", end=" ")
    if len(app_environmentVariables) != 0:
        print("OK")
        for i in range(len(app_environmentVariables)):
            app_environmentVariables_name.append(app_environmentVariables[i]["name"])
            app_environmentVariables_value.append(app_environmentVariables[i]["value"])
        app_environmentVariables_dict = dict(zip(app_environmentVariables_name, app_environmentVariables_value))
        for k,v in app_environmentVariables_dict.items():
            print("> ", k, " : ", v) 
else:
    print("NULL")

print("Updating {new_app}...", end=" ")
if len(api_call_results) != 0:
    print("OK")
    print("Updating EnvironmentVariables...", end=" ")
    if len(app_environmentVariables_dict) != 0:
        print("OK")
        for key, value in app_environmentVariables_dict.items():
            if key not in app_environmentVariables_filter:
                query_string = """
                    mutation {{
                        addEnvironmentVariable(
                            name: "{key}",
                            value: "{value}",
                            appname: "{new_app}"
                        ){{
                            ok
                            error
                        }}
                    }}
                    """.format(key=key, value=value, new_app=new_app)
                # client.execute(gql(query_string))
                print(">  {key} : {value}".format(key=key, value=value))
    else:
        print("NULL")
    print("Updating LinkedServices...", end=" ")
    if len(app_linkedServices) != 0:
        print("OK")
        filtered_dict = dict()
        for i in app_linkedServices_serviceType:
            query_string = """
                addService(serviceType: {service_type}, name: "{new_app}-{service_type}") {{
                service {{
                    name
                    serviceType
                    created
                }}
                error
            }}  
            """.format(service_type=i, new_app=new_app)
            # client.execute(gql(query_string))
            print("Adding linkedServices: {service_type}, {new_app}-{service_type}".format(service_type=i, new_app=new_app))
    else:
        print("NULL")

    print("Updating Mounts...", end=" ")
    if len(app_mounts_dict) !=0:
        print("OK")
        for key, value in app_mounts_dict.items():
            query_string = """
                mutation {{
                    mountDirectory(
                        hostDir: "{app_mounts_hostDir}",
                        targetDir: "{app_mounts_targetDir}", 
                        appname: "{new_app}"
                        ) {{
                            ok
                            error
                        }}
                }}
                """.format(app_mounts_hostDir=key, app_mounts_targetDir=value, new_app=new_app)
            # client.execute(gql(query_string))
            print("Adding mounts...DONE!")

    else:
        print("NULL")


In [ ]:
api_call

In [ ]:
dir(api_call)

In [ ]:
# # addApp
# mutation {
# 	addApp(name: "") {
# 		error
#     app {
#       name
#       urlOnServer
#       thumbnailUrl
#       hasWorkspace
#     }
#   }
# }

addApp = {
  "data": {
    "addApp": {
      "error": "Invalid app name. Names should be between 3 to 30 characters long, start with a letter, and only contain lower case letters, numbers, and -",
      "app": null
    }
  }
}

# # deleteApp
# mutation {
# 	deleteApp(name: "") {
#     error
#     ok
#   }
# }

deleteApp ={
  "data": {
    "deleteApp": {
      "error": "App does not exist.",
      "ok": false
    }
  }
}

# addEnvironmentVariable
addEnvironmentVariable = {
  "data": {
    "addEnvironmentVariable": {
      "error": "App does not exist.",
      "refresh": false,
      "ok": false
    }
  }
}

# addService
addService = {
  "data": {
    "addService": {
      "error": "Invalid service name. Names should be between 3 to 30 characters long, start with a letter, and only contain lower case letters, numbers, and -, with no spacing"
    }
  }

  {
  "data": {
    "addService": {
      "error": "A service with the given name already exists. Please choose a different name."
    }
  }
}

# mutation { 
# 	mountDirectory(appname: "aa-chris") {
#   	error
#   }
# }

{
  "data": {
    "mountDirectory": {
      "error": "App does not exist."
    }
  }
}

{
  "data": {
    "mountDirectory": {
      "error": "Only directories specified in the Allowed Directories for Mapping list of the Dash Enterprise configuration can be mapped to Dash apps. Please ask your administrator to add None to the list as shown in the documentation (https://dash.plot.ly/dash-enterprise/map-local-directories) and then try again."
    }
  }
}

In [32]:
import random
import string

    # dash_app_name = "review-app-" + "".join(random.choices(string.ascii_uppercase + string.digits, k=6))"".join(random.choices(string.ascii_uppercase + string.digits, k=6)

for i in range(10):
    dash_app_name = "review-app-" + "".join(random.choices(string.ascii_uppercase + string.digits, k=6))
    print(dash_app_name)

review-app-YN715P
review-app-JIGHR2
review-app-W9E1B4
review-app-4PUGD1
review-app-HXO4OA
review-app-YSHAGR
review-app-JTA9R7
review-app-R6AP6A
review-app-BM8N9S
review-app-9DKBXB


In [38]:
adds = "strings"

badds = ("The bard plucked the {adds} of their lute as they gazed into the maiden's eyes".format(adds=adds))

In [41]:
my_dict = {'apple': None}

print(my_dict['apple'])

None


In [46]:
type(my_dict['apple']) == 

SyntaxError: invalid syntax (<ipython-input-46-beef1a11a588>, line 1)

In [54]:
value = None


if value is None and 1 < 2 and type(value) == None:
    print(1)


In [58]:
type(value) == None

if value is not None:
    print(1)
print(0)

0
